In [1]:
import numpy as np
import pandas as pd
from numpy import random
import sqlite3

import re

import random

import warnings
warnings.filterwarnings('ignore')

seed = 1998

In [2]:
area_list = ['Dobong', 'Dongdaemun', 'Dongjag', 'eunpyeong', 'geumcheon',
'Guro', 'GwanUck', 'Jongno', 'Junggu', 'Jungnang', 'Kongbuk',
'Kongdong', 'Kongjin', 'Kongnam', 'Kongseo', 'Mapo', 'Nowon', 'Seocho',
'Seodaemun', 'Seongbuk', 'Seongdong', 'songpa', 'Yangcheon',
'Yeongdeungpo', 'Yongsan']

len(area_list)

25

In [3]:
def solve(x):
    a = re.findall(r'\d{4}년 \d{1,2}월 \d{1,2}일', str(x))
    try: return a[0]
    except: return np.NAN

data = pd.DataFrame()

for area in area_list:
    tem = pd.read_csv(f'./user/{area}.csv')
    tem['userid'] = tem['link'].apply(lambda x: str(re.findall(r'my/[a-z0-9]+/', x))[5:-3])
    tem['date'] = tem['user'].apply(lambda x: solve(x))

    tem = tem[['userid', 'rest', 'date']]
    tem = tem.drop_duplicates()
    data = pd.concat([data, tem])

data = data.drop_duplicates() # 30만개정도 빠짐
data = data[~data['rest'].isnull()] # 7개정도 null 값이 있더라
data.reset_index(drop=True, inplace = True)

In [4]:
rest = pd.read_csv('rest.csv')
rest = rest[['url', 'rest_code']]

user = pd.read_csv('user.csv')
user = user[['user','user_code']]

In [17]:
data = data[['userid', 'rest', 'date']]

In [18]:
data = pd.merge(data, rest, left_on = 'rest', right_on = 'url', how = 'left')
data = data[~data['rest_code'].isnull()]
data['rest_code'] = data['rest_code'].astype('int')
data

,userid,rest,date,url,rest_code
0,61836eb882ac4fbb28ec8614,3.144232e+07,2022년 12월 6일,31442321,2010
1,5b9df8bcce3e9302c4c6e455,3.144232e+07,2022년 10월 3일,31442321,2010
2,5fe311dbc6045131b90d40e2,3.144232e+07,2023년 1월 8일,31442321,2010
3,5c9645b66d633b3d2257e846,3.144232e+07,2022년 12월 10일,31442321,2010
4,5b9df8bcce3e9302c4c6e455,3.144232e+07,2022년 8월 21일,31442321,2010
...,...,...,...,...,...
20637519,5da155d88f87a842bc3d0fdd,1.866837e+09,2021년 3월 13일,1866836918,33095
20637520,5bd536b29a1bb1c4c8c359e6,1.866837e+09,2021년 4월 4일,1866836918,33095
20637521,5f06018dc6c911a490bbd9e7,1.866837e+09,2021년 7월 20일,1866836918,33095
20637522,5e84795b8f87a842bc34a38c,1.866837e+09,2021년 1월 28일,1866836918,33095


In [21]:
data = pd.merge(data, user, left_on = 'userid', right_on = 'user', how = 'left')
data['user_code'] = data['user_code'].fillna(-1)
data['user_code'] = data['user_code'].astype('int')

,userid,rest,date,url,rest_code,user,user_code
0,61836eb882ac4fbb28ec8614,3.144232e+07,2022년 12월 6일,31442321,2010,61836eb882ac4fbb28ec8614,371776.0
1,5b9df8bcce3e9302c4c6e455,3.144232e+07,2022년 10월 3일,31442321,2010,5b9df8bcce3e9302c4c6e455,25720.0
2,5fe311dbc6045131b90d40e2,3.144232e+07,2023년 1월 8일,31442321,2010,5fe311dbc6045131b90d40e2,343010.0
3,5c9645b66d633b3d2257e846,3.144232e+07,2022년 12월 10일,31442321,2010,5c9645b66d633b3d2257e846,104286.0
4,5b9df8bcce3e9302c4c6e455,3.144232e+07,2022년 8월 21일,31442321,2010,5b9df8bcce3e9302c4c6e455,25720.0
...,...,...,...,...,...,...,...
20637519,5da155d88f87a842bc3d0fdd,1.866837e+09,2021년 3월 13일,1866836918,33095,5da155d88f87a842bc3d0fdd,162236.0
20637520,5bd536b29a1bb1c4c8c359e6,1.866837e+09,2021년 4월 4일,1866836918,33095,5bd536b29a1bb1c4c8c359e6,39295.0
20637521,5f06018dc6c911a490bbd9e7,1.866837e+09,2021년 7월 20일,1866836918,33095,5f06018dc6c911a490bbd9e7,306288.0
20637522,5e84795b8f87a842bc34a38c,1.866837e+09,2021년 1월 28일,1866836918,33095,5e84795b8f87a842bc34a38c,238823.0


In [32]:
data = data.drop_duplicates(['userid','rest'])

In [ ]:
data

In [30]:
data = data[['userid', 'rest', 'date', 'rest_code', 'user_code']].reset_index(drop = True)
data = data.drop_duplicates(['userid','rest'])
data.to_csv('../../airflow/data/data_all.csv', index = False)

In [33]:
data.to_csv('../../airflow/data/data_all2.csv', index = False)

In [7]:
rest2idx = rest.set_index('url')

def make_restcode(x):
    try:
        tem = rest2idx.loc[x]
        return tem
    except:
        return -1

data['rest_code'] = data['rest'].apply(lambda x : make_restcode(x))
data

In [ ]:
user2idx = user.set_index('user')

def make_usercode(x):
    try:
        tem = user2idx.loc[x]
        return tem
    except:
        return -1

data['user_code'] = data['userid'].apply(lambda x : make_usercode(x))
data

In [ ]:
data